# Module 7: Indexing for Performance

## 🎯 Learning Objectives
By the end of this module, you will:
- Understand different indexing algorithms and their trade-offs
- Configure indexes for optimal performance in production scenarios
- Balance speed, accuracy, and memory usage based on requirements
- Know when and how to rebuild indexes for optimal performance
- Implement and tune HNSW, IVF, and other indexing algorithms

## 📚 Key Concepts

### Why Indexing Matters

Without proper indexing, vector similarity search becomes a **linear scan problem**:

```python
# Naive approach: O(n) complexity
def linear_search(query_vector, all_vectors):
    similarities = []
    for vector in all_vectors:  # Check EVERY vector
        similarity = cosine_similarity(query_vector, vector)
        similarities.append(similarity)
    return sorted(similarities, reverse=True)[:k]

# With 1M vectors: ~1M similarity calculations per query!
```

**The Problem**: With millions of vectors, this becomes prohibitively slow
**The Solution**: Indexing algorithms that provide sub-linear search complexity

### 🏗️ Major Indexing Algorithms

#### 1. HNSW (Hierarchical Navigable Small World) 🏆
- **Best overall performance** for most use cases
- **Graph-based approach**: Builds a multi-layer graph structure
- **Complexity**: O(log n) expected search time
- **Memory**: ~1.1x original embedding size

#### 2. IVF (Inverted File Index)
- **Clustering-based approach**: Groups similar vectors together
- **Good for large datasets** with moderate accuracy requirements
- **Complexity**: O(sqrt(n)) search time
- **Memory**: More memory-efficient than HNSW

#### 3. LSH (Locality Sensitive Hashing)
- **Hash-based approach**: Maps similar vectors to same buckets
- **Fast but approximate**: Trade accuracy for speed
- **Best for**: Extremely large datasets where speed > accuracy

#### 4. Product Quantization (PQ)
- **Compression technique**: Reduces memory usage by 8-32x
- **Often combined** with HNSW or IVF
- **Trade-off**: Significant memory savings vs some accuracy loss

### 2025 Performance Landscape 📊

| Algorithm | Query Speed | Memory Usage | Accuracy | Build Time |
|-----------|-------------|--------------|----------|------------|
| **HNSW** | Excellent | Moderate | Excellent | Fast |
| **IVF-HNSW** | Very Good | Low | Very Good | Moderate |
| **PQ-HNSW** | Good | Very Low | Good | Fast |
| **LSH** | Excellent | Low | Moderate | Very Fast |

### Performance Trade-offs

Every indexing decision involves **fundamental trade-offs**:

1. **Speed vs Accuracy**: Faster algorithms often sacrifice some precision
2. **Memory vs Performance**: More memory usually means faster queries
3. **Build Time vs Query Time**: Complex indexes take longer to build but query faster
4. **Update Frequency vs Optimization**: Frequent updates may require different strategies

## 🛠️ Setup
Let's install the required packages and set up our indexing performance lab.

In [ ]:
# Install required packages
!pip install -q faiss-cpu hnswlib qdrant-client sentence-transformers numpy pandas matplotlib seaborn
!pip install -q scikit-learn psutil memory-profiler time-memory
# Note: For GPU acceleration, replace faiss-cpu with faiss-gpu

In [ ]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from typing import List, Dict, Any, Tuple, Optional
import json
import psutil
import gc
from memory_profiler import memory_usage

# Indexing libraries
import faiss
import hnswlib
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct, HnswConfig

# ML libraries
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.random_projection import SparseRandomProjection

# Set up visualization
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

print("✅ Setup complete!")
print(f"📅 Today's date: {datetime.now().strftime('%Y-%m-%d')}")
print(f"💾 Available RAM: {psutil.virtual_memory().total / (1024**3):.1f} GB")

## 🧪 Exercise 1: Understanding Indexing Algorithms

Let's implement and compare different indexing algorithms to understand their characteristics.

In [ ]:
class IndexingAlgorithmComparison:
    """Compare different indexing algorithms for vector similarity search"""
    
    def __init__(self):
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
        self.indexes = {}
        self.build_times = {}
        self.memory_usage = {}
        
    def generate_test_data(self, num_vectors: int = 10000, seed: int = 42) -> Tuple[np.ndarray, List[str]]:
        """Generate synthetic test data for indexing experiments"""
        np.random.seed(seed)
        
        # Create diverse text samples
        topics = [
            "machine learning and artificial intelligence",
            "data science and statistical analysis", 
            "computer vision and image processing",
            "natural language processing and linguistics",
            "web development and software engineering",
            "database systems and data storage",
            "cloud computing and distributed systems",
            "cybersecurity and network protection",
            "mobile app development and user interfaces",
            "scientific computing and numerical methods"
        ]
        
        documents = []
        for i in range(num_vectors):
            topic = topics[i % len(topics)]
            variation = np.random.randint(1, 100)
            doc = f"Document {i} about {topic}, covering advanced concepts and practical applications in the field. Variation {variation}."
            documents.append(doc)
        
        print(f"Generating embeddings for {num_vectors} documents...")
        embeddings = self.embedding_model.encode(documents, show_progress_bar=True)
        
        # Normalize embeddings for cosine similarity
        embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
        
        return embeddings.astype('float32'), documents
    
    def build_linear_index(self, embeddings: np.ndarray, name: str = "linear") -> Dict[str, Any]:
        """Build a linear (brute-force) search index for baseline comparison"""
        start_time = time.time()
        
        # For linear search, we just store the embeddings
        index = {
            'type': 'linear',
            'embeddings': embeddings,
            'size': len(embeddings)
        }
        
        build_time = time.time() - start_time
        memory_size = embeddings.nbytes
        
        self.indexes[name] = index
        self.build_times[name] = build_time
        self.memory_usage[name] = memory_size
        
        print(f"✅ Built {name} index: {len(embeddings):,} vectors in {build_time:.2f}s")
        return index
    
    def build_hnsw_index(self, embeddings: np.ndarray, name: str = "hnsw", 
                        ef_construction: int = 200, M: int = 16) -> hnswlib.Index:
        """Build HNSW index using hnswlib"""
        start_time = time.time()
        
        # Initialize HNSW index
        dim = embeddings.shape[1]
        index = hnswlib.Index(space='cosine', dim=dim)
        
        # Configure HNSW parameters
        index.init_index(
            max_elements=len(embeddings),
            ef_construction=ef_construction,
            M=M
        )
        
        # Add vectors to index
        labels = np.arange(len(embeddings))
        index.add_items(embeddings, labels)
        
        build_time = time.time() - start_time
        
        # Estimate memory usage (approximate)
        memory_size = embeddings.nbytes * 1.1  # HNSW adds ~10% overhead
        
        self.indexes[name] = index
        self.build_times[name] = build_time
        self.memory_usage[name] = memory_size
        
        print(f"✅ Built {name} index: {len(embeddings):,} vectors in {build_time:.2f}s")
        print(f"   Parameters: ef_construction={ef_construction}, M={M}")
        return index
    
    def build_ivf_index(self, embeddings: np.ndarray, name: str = "ivf", 
                       nlist: int = 100) -> faiss.IndexIVFFlat:
        """Build IVF (Inverted File) index using FAISS"""
        start_time = time.time()
        
        dim = embeddings.shape[1]
        
        # Create quantizer (for clustering)
        quantizer = faiss.IndexFlatIP(dim)  # Inner product for cosine similarity
        
        # Create IVF index
        index = faiss.IndexIVFFlat(quantizer, dim, nlist)
        
        # Train the index (clustering step)
        index.train(embeddings)
        
        # Add vectors to index
        index.add(embeddings)
        
        build_time = time.time() - start_time
        
        # Estimate memory usage
        memory_size = embeddings.nbytes * 1.05  # IVF adds ~5% overhead
        
        self.indexes[name] = index
        self.build_times[name] = build_time
        self.memory_usage[name] = memory_size
        
        print(f"✅ Built {name} index: {len(embeddings):,} vectors in {build_time:.2f}s")
        print(f"   Parameters: nlist={nlist} clusters")
        return index
    
    def build_lsh_index(self, embeddings: np.ndarray, name: str = "lsh", 
                       n_components: int = 64) -> Dict[str, Any]:
        """Build LSH (Locality Sensitive Hashing) index using random projections"""
        start_time = time.time()
        
        # Create random projection transformer
        lsh_transformer = SparseRandomProjection(
            n_components=n_components,
            random_state=42
        )
        
        # Transform embeddings to hash space
        hash_embeddings = lsh_transformer.fit_transform(embeddings)
        
        # Binarize the hash values
        binary_hashes = (hash_embeddings > 0).astype(np.int8)
        
        index = {
            'type': 'lsh',
            'transformer': lsh_transformer,
            'binary_hashes': binary_hashes,
            'original_embeddings': embeddings,
            'n_components': n_components
        }
        
        build_time = time.time() - start_time
        
        # Memory usage: binary hashes + transformer
        memory_size = binary_hashes.nbytes + embeddings.nbytes * 0.1
        
        self.indexes[name] = index
        self.build_times[name] = build_time
        self.memory_usage[name] = memory_size
        
        print(f"✅ Built {name} index: {len(embeddings):,} vectors in {build_time:.2f}s")
        print(f"   Parameters: {n_components} hash components")
        return index
    
    def search_linear(self, query_embedding: np.ndarray, index: Dict[str, Any], k: int = 10) -> Tuple[List[int], List[float]]:
        """Search using linear scan"""
        embeddings = index['embeddings']
        
        # Calculate cosine similarities
        similarities = np.dot(embeddings, query_embedding.flatten())
        
        # Get top-k indices
        top_indices = np.argsort(similarities)[::-1][:k]
        top_scores = similarities[top_indices]
        
        return top_indices.tolist(), top_scores.tolist()
    
    def search_hnsw(self, query_embedding: np.ndarray, index: hnswlib.Index, 
                   k: int = 10, ef: int = 50) -> Tuple[List[int], List[float]]:
        """Search using HNSW index"""
        # Set search parameter
        index.set_ef(ef)
        
        # Search
        labels, distances = index.knn_query(query_embedding, k=k)
        
        # Convert distances to similarities (cosine distance -> cosine similarity)
        similarities = 1 - distances[0]
        
        return labels[0].tolist(), similarities.tolist()
    
    def search_ivf(self, query_embedding: np.ndarray, index: faiss.IndexIVFFlat, 
                  k: int = 10, nprobe: int = 10) -> Tuple[List[int], List[float]]:
        """Search using IVF index"""
        # Set search parameter
        index.nprobe = nprobe
        
        # Search
        similarities, indices = index.search(query_embedding.reshape(1, -1), k)
        
        return indices[0].tolist(), similarities[0].tolist()
    
    def search_lsh(self, query_embedding: np.ndarray, index: Dict[str, Any], 
                  k: int = 10) -> Tuple[List[int], List[float]]:
        """Search using LSH index"""
        # Transform query to hash space
        query_hash = index['transformer'].transform(query_embedding.reshape(1, -1))
        query_binary = (query_hash > 0).astype(np.int8)
        
        # Calculate Hamming distances to all binary hashes
        hamming_distances = np.sum(index['binary_hashes'] != query_binary, axis=1)
        
        # Get candidates with smallest Hamming distances
        candidate_indices = np.argsort(hamming_distances)[:k*5]  # Get more candidates
        
        # Refine with actual cosine similarity
        candidate_embeddings = index['original_embeddings'][candidate_indices]
        similarities = np.dot(candidate_embeddings, query_embedding.flatten())
        
        # Get final top-k
        top_k_among_candidates = np.argsort(similarities)[::-1][:k]
        final_indices = candidate_indices[top_k_among_candidates]
        final_scores = similarities[top_k_among_candidates]
        
        return final_indices.tolist(), final_scores.tolist()

# Initialize the indexing comparison
indexing_lab = IndexingAlgorithmComparison()
print("🏗️ Indexing Algorithm Comparison Lab initialized!")

In [ ]:
# Generate test data
print("📊 Generating test dataset...")
embeddings, documents = indexing_lab.generate_test_data(num_vectors=5000)

print(f"✅ Generated {len(embeddings):,} embeddings")
print(f"📏 Embedding dimension: {embeddings.shape[1]}")
print(f"💾 Dataset size: {embeddings.nbytes / (1024**2):.1f} MB")

# Display sample documents
print("\n📄 Sample documents:")
for i in range(3):
    print(f"   {i+1}. {documents[i]}")

In [ ]:
# Build different indexes
print("🏗️ BUILDING INDEXES")
print("=" * 50)

# 1. Linear (baseline)
print("\n1. Building Linear Index (baseline):")
linear_index = indexing_lab.build_linear_index(embeddings)

# 2. HNSW with default parameters
print("\n2. Building HNSW Index:")
hnsw_index = indexing_lab.build_hnsw_index(embeddings, "hnsw_default", ef_construction=200, M=16)

# 3. HNSW with high-performance parameters
print("\n3. Building HNSW Index (High Performance):")
hnsw_fast_index = indexing_lab.build_hnsw_index(embeddings, "hnsw_fast", ef_construction=100, M=8)

# 4. HNSW with high-accuracy parameters
print("\n4. Building HNSW Index (High Accuracy):")
hnsw_accurate_index = indexing_lab.build_hnsw_index(embeddings, "hnsw_accurate", ef_construction=400, M=32)

# 5. IVF Index
print("\n5. Building IVF Index:")
ivf_index = indexing_lab.build_ivf_index(embeddings, "ivf", nlist=100)

# 6. LSH Index
print("\n6. Building LSH Index:")
lsh_index = indexing_lab.build_lsh_index(embeddings, "lsh", n_components=64)

print("\n✅ All indexes built successfully!")

In [ ]:
# Compare build times and memory usage
print("📊 INDEX BUILD COMPARISON")
print("=" * 60)

build_comparison = []
for name in indexing_lab.build_times.keys():
    build_comparison.append({
        'Index': name.replace('_', ' ').title(),
        'Build Time (s)': indexing_lab.build_times[name],
        'Memory Usage (MB)': indexing_lab.memory_usage[name] / (1024**2),
        'Memory Ratio': indexing_lab.memory_usage[name] / embeddings.nbytes
    })

build_df = pd.DataFrame(build_comparison)
print(build_df.to_string(index=False, float_format='%.3f'))

# Visualize build comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Build time comparison
bars1 = ax1.bar(range(len(build_df)), build_df['Build Time (s)'])
ax1.set_title('Index Build Time Comparison', fontweight='bold', fontsize=14)
ax1.set_ylabel('Build Time (seconds)')
ax1.set_xlabel('Index Type')
ax1.set_xticks(range(len(build_df)))
ax1.set_xticklabels(build_df['Index'], rotation=45, ha='right')

# Add value labels
for i, bar in enumerate(bars1):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{height:.3f}s', ha='center', va='bottom', fontweight='bold')

# Memory usage comparison
bars2 = ax2.bar(range(len(build_df)), build_df['Memory Ratio'])
ax2.set_title('Memory Usage Ratio', fontweight='bold', fontsize=14)
ax2.set_ylabel('Memory Ratio (vs Original)')
ax2.set_xlabel('Index Type')
ax2.set_xticks(range(len(build_df)))
ax2.set_xticklabels(build_df['Index'], rotation=45, ha='right')
ax2.axhline(y=1.0, color='red', linestyle='--', alpha=0.7, label='Original Size')
ax2.legend()

# Add value labels
for i, bar in enumerate(bars2):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{height:.2f}x', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n💡 Build Time Insights:")
print("- Linear 'index' is just storing embeddings (no actual indexing)")
print("- HNSW build time increases with ef_construction and M parameters")
print("- IVF requires clustering step, making it slower to build")
print("- LSH is fastest to build but uses additional hash storage")

## ⚡ Exercise 2: Query Performance Benchmarking

Let's benchmark query performance across different indexes and parameters.

In [ ]:
class QueryPerformanceBenchmark:
    """Benchmark query performance across different indexing algorithms"""
    
    def __init__(self, indexing_lab: IndexingAlgorithmComparison):
        self.indexing_lab = indexing_lab
        self.query_results = []
    
    def generate_test_queries(self, num_queries: int = 100) -> np.ndarray:
        """Generate test queries for benchmarking"""
        query_texts = [
            "machine learning algorithms and neural networks",
            "data science statistical analysis and visualization",
            "computer vision image processing techniques",
            "natural language processing and text analysis",
            "web development and software architecture",
            "database optimization and data storage",
            "cloud computing distributed systems",
            "cybersecurity and network protection",
            "mobile application development frameworks",
            "scientific computing numerical methods"
        ]
        
        # Create test queries by cycling through topics
        test_queries = [query_texts[i % len(query_texts)] for i in range(num_queries)]
        
        # Generate embeddings for queries
        query_embeddings = self.indexing_lab.embedding_model.encode(test_queries)
        query_embeddings = query_embeddings / np.linalg.norm(query_embeddings, axis=1, keepdims=True)
        
        return query_embeddings.astype('float32')
    
    def benchmark_index(self, index_name: str, query_embeddings: np.ndarray, 
                       k: int = 10, num_warmup: int = 5) -> Dict[str, Any]:
        """Benchmark a specific index"""
        index = self.indexing_lab.indexes[index_name]
        query_times = []
        all_results = []
        
        # Warmup queries
        for i in range(min(num_warmup, len(query_embeddings))):
            self._search_index(index_name, query_embeddings[i], k)
        
        # Actual benchmark
        for query_emb in query_embeddings:
            start_time = time.time()
            indices, scores = self._search_index(index_name, query_emb, k)
            query_time = time.time() - start_time
            
            query_times.append(query_time * 1000)  # Convert to milliseconds
            all_results.append((indices, scores))
        
        return {
            'index_name': index_name,
            'query_times': query_times,
            'avg_time': np.mean(query_times),
            'p50_time': np.percentile(query_times, 50),
            'p95_time': np.percentile(query_times, 95),
            'p99_time': np.percentile(query_times, 99),
            'queries_per_second': 1000 / np.mean(query_times),
            'results': all_results
        }
    
    def _search_index(self, index_name: str, query_embedding: np.ndarray, k: int) -> Tuple[List[int], List[float]]:
        """Search a specific index based on its type"""
        index = self.indexing_lab.indexes[index_name]
        
        if index_name == "linear":
            return self.indexing_lab.search_linear(query_embedding, index, k)
        elif "hnsw" in index_name:
            # Different ef values for different HNSW variants
            ef = 200 if "accurate" in index_name else (50 if "fast" in index_name else 100)
            return self.indexing_lab.search_hnsw(query_embedding, index, k, ef)
        elif index_name == "ivf":
            return self.indexing_lab.search_ivf(query_embedding, index, k, nprobe=10)
        elif index_name == "lsh":
            return self.indexing_lab.search_lsh(query_embedding, index, k)
        else:
            raise ValueError(f"Unknown index type: {index_name}")
    
    def calculate_recall(self, ground_truth_results: List[Tuple], test_results: List[Tuple], k: int = 10) -> float:
        """Calculate recall@k between two result sets"""
        if len(ground_truth_results) != len(test_results):
            raise ValueError("Result sets must have same length")
        
        total_recall = 0.0
        
        for gt_result, test_result in zip(ground_truth_results, test_results):
            gt_indices = set(gt_result[0][:k])  # Ground truth top-k indices
            test_indices = set(test_result[0][:k])  # Test result top-k indices
            
            # Calculate recall for this query
            intersection = len(gt_indices.intersection(test_indices))
            recall = intersection / len(gt_indices)
            total_recall += recall
        
        return total_recall / len(ground_truth_results)
    
    def run_comprehensive_benchmark(self, num_queries: int = 50) -> pd.DataFrame:
        """Run comprehensive benchmark across all indexes"""
        print(f"🚀 QUERY PERFORMANCE BENCHMARK")
        print(f"Running {num_queries} queries across all indexes...")
        print("=" * 60)
        
        # Generate test queries
        query_embeddings = self.generate_test_queries(num_queries)
        
        benchmark_results = []
        ground_truth_results = None
        
        # Benchmark each index
        for index_name in self.indexing_lab.indexes.keys():
            print(f"\nBenchmarking {index_name}...")
            
            result = self.benchmark_index(index_name, query_embeddings)
            
            # Use linear search as ground truth for recall calculation
            if index_name == "linear":
                ground_truth_results = result['results']
                recall = 1.0  # Perfect recall for ground truth
            else:
                recall = self.calculate_recall(ground_truth_results, result['results'])
            
            benchmark_results.append({
                'Index': index_name.replace('_', ' ').title(),
                'Avg Latency (ms)': result['avg_time'],
                'P50 Latency (ms)': result['p50_time'],
                'P95 Latency (ms)': result['p95_time'],
                'P99 Latency (ms)': result['p99_time'],
                'Queries/sec': result['queries_per_second'],
                'Recall@10': recall,
                'Speedup vs Linear': benchmark_results[0]['Avg Latency (ms)'] / result['avg_time'] if benchmark_results else 1.0
            })
            
            print(f"   Avg latency: {result['avg_time']:.2f}ms")
            print(f"   P95 latency: {result['p95_time']:.2f}ms")
            print(f"   Queries/sec: {result['queries_per_second']:.1f}")
            print(f"   Recall@10: {recall:.3f}")
        
        # Fix speedup calculation
        linear_time = benchmark_results[0]['Avg Latency (ms)']
        for result in benchmark_results:
            result['Speedup vs Linear'] = linear_time / result['Avg Latency (ms)']
        
        return pd.DataFrame(benchmark_results)

# Initialize benchmark
query_benchmark = QueryPerformanceBenchmark(indexing_lab)
print("⚡ Query Performance Benchmark initialized!")

In [ ]:
# Run comprehensive benchmark
performance_results = query_benchmark.run_comprehensive_benchmark(num_queries=30)

print("\n📊 PERFORMANCE RESULTS SUMMARY")
print("=" * 80)
print(performance_results.to_string(index=False, float_format='%.3f'))

# Visualize performance results
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# 1. Average latency comparison
bars1 = ax1.bar(range(len(performance_results)), performance_results['Avg Latency (ms)'])
ax1.set_title('Average Query Latency', fontweight='bold', fontsize=14)
ax1.set_ylabel('Latency (milliseconds)')
ax1.set_xlabel('Index Type')
ax1.set_xticks(range(len(performance_results)))
ax1.set_xticklabels(performance_results['Index'], rotation=45, ha='right')
ax1.set_yscale('log')

# Add value labels
for i, bar in enumerate(bars1):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height * 1.1,
             f'{height:.2f}ms', ha='center', va='bottom', fontweight='bold', fontsize=9)

# 2. Queries per second
bars2 = ax2.bar(range(len(performance_results)), performance_results['Queries/sec'])
ax2.set_title('Query Throughput', fontweight='bold', fontsize=14)
ax2.set_ylabel('Queries per Second')
ax2.set_xlabel('Index Type')
ax2.set_xticks(range(len(performance_results)))
ax2.set_xticklabels(performance_results['Index'], rotation=45, ha='right')

# Add value labels
for i, bar in enumerate(bars2):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
             f'{height:.0f}', ha='center', va='bottom', fontweight='bold', fontsize=9)

# 3. Recall vs Speed trade-off
ax3.scatter(performance_results['Avg Latency (ms)'], performance_results['Recall@10'], 
           s=100, alpha=0.7)
ax3.set_title('Recall vs Speed Trade-off', fontweight='bold', fontsize=14)
ax3.set_xlabel('Average Latency (ms)')
ax3.set_ylabel('Recall@10')
ax3.set_xscale('log')
ax3.grid(True, alpha=0.3)

# Add labels for each point
for i, row in performance_results.iterrows():
    ax3.annotate(row['Index'], 
                (row['Avg Latency (ms)'], row['Recall@10']),
                xytext=(5, 5), textcoords='offset points', fontsize=8)

# 4. Speedup comparison
bars4 = ax4.bar(range(len(performance_results)), performance_results['Speedup vs Linear'])
ax4.set_title('Speedup vs Linear Search', fontweight='bold', fontsize=14)
ax4.set_ylabel('Speedup Factor')
ax4.set_xlabel('Index Type')
ax4.set_xticks(range(len(performance_results)))
ax4.set_xticklabels(performance_results['Index'], rotation=45, ha='right')
ax4.axhline(y=1.0, color='red', linestyle='--', alpha=0.7, label='Linear Baseline')
ax4.legend()

# Add value labels
for i, bar in enumerate(bars4):
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height + 0.1,
             f'{height:.1f}x', ha='center', va='bottom', fontweight='bold', fontsize=9)

plt.tight_layout()
plt.show()

print("\n🎯 Key Performance Insights:")
print("- HNSW variants provide the best balance of speed and accuracy")
print("- LSH offers extreme speed but with reduced accuracy")
print("- IVF provides good performance for large-scale scenarios")
print("- Parameter tuning significantly impacts HNSW performance")
print("- All approximate methods achieve 10x+ speedup over linear search")

## ⚙️ Exercise 3: Parameter Tuning and Optimization

Let's explore how parameter tuning affects performance and accuracy.

In [ ]:
class ParameterTuningLab:
    """Laboratory for parameter tuning experiments"""
    
    def __init__(self, embeddings: np.ndarray, documents: List[str]):
        self.embeddings = embeddings
        self.documents = documents
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
        
    def tune_hnsw_parameters(self) -> pd.DataFrame:
        """Systematically tune HNSW parameters"""
        print("🔧 HNSW PARAMETER TUNING")
        print("=" * 50)
        
        # Parameter combinations to test
        ef_construction_values = [100, 200, 400]
        M_values = [8, 16, 32]
        ef_search_values = [50, 100, 200]
        
        # Generate test queries
        test_queries = [
            "machine learning algorithms",
            "data science methods",
            "computer vision techniques"
        ]
        query_embeddings = self.embedding_model.encode(test_queries)
        query_embeddings = query_embeddings / np.linalg.norm(query_embeddings, axis=1, keepdims=True)
        
        # Build ground truth with linear search
        ground_truth_results = []
        for query_emb in query_embeddings:
            similarities = np.dot(self.embeddings, query_emb)
            top_indices = np.argsort(similarities)[::-1][:10]
            ground_truth_results.append(top_indices.tolist())
        
        tuning_results = []
        
        for ef_construction in ef_construction_values:
            for M in M_values:
                print(f"\nTesting ef_construction={ef_construction}, M={M}")
                
                # Build index with these parameters
                build_start = time.time()
                
                index = hnswlib.Index(space='cosine', dim=self.embeddings.shape[1])
                index.init_index(
                    max_elements=len(self.embeddings),
                    ef_construction=ef_construction,
                    M=M
                )
                index.add_items(self.embeddings, np.arange(len(self.embeddings)))
                
                build_time = time.time() - build_start
                
                # Test different ef_search values
                for ef_search in ef_search_values:
                    index.set_ef(ef_search)
                    
                    # Measure query performance
                    query_times = []
                    recalls = []
                    
                    for i, query_emb in enumerate(query_embeddings):
                        start_time = time.time()
                        labels, distances = index.knn_query(query_emb, k=10)
                        query_time = time.time() - start_time
                        
                        query_times.append(query_time * 1000)  # Convert to ms
                        
                        # Calculate recall
                        predicted_indices = set(labels[0])
                        true_indices = set(ground_truth_results[i])
                        recall = len(predicted_indices.intersection(true_indices)) / len(true_indices)
                        recalls.append(recall)
                    
                    avg_query_time = np.mean(query_times)
                    avg_recall = np.mean(recalls)
                    
                    tuning_results.append({
                        'ef_construction': ef_construction,
                        'M': M,
                        'ef_search': ef_search,
                        'build_time': build_time,
                        'avg_query_time': avg_query_time,
                        'avg_recall': avg_recall,
                        'queries_per_second': 1000 / avg_query_time
                    })
                    
                    print(f"   ef_search={ef_search}: {avg_query_time:.2f}ms, recall={avg_recall:.3f}")
        
        return pd.DataFrame(tuning_results)
    
    def tune_ivf_parameters(self) -> pd.DataFrame:
        """Tune IVF parameters"""
        print("\n🔧 IVF PARAMETER TUNING")
        print("=" * 50)
        
        # Parameter combinations
        nlist_values = [50, 100, 200, 400]
        nprobe_values = [5, 10, 20, 40]
        
        # Generate test queries
        test_queries = [
            "machine learning algorithms",
            "data science methods",
            "computer vision techniques"
        ]
        query_embeddings = self.embedding_model.encode(test_queries)
        query_embeddings = query_embeddings / np.linalg.norm(query_embeddings, axis=1, keepdims=True)
        
        # Build ground truth
        ground_truth_results = []
        for query_emb in query_embeddings:
            similarities = np.dot(self.embeddings, query_emb)
            top_indices = np.argsort(similarities)[::-1][:10]
            ground_truth_results.append(top_indices.tolist())
        
        tuning_results = []
        
        for nlist in nlist_values:
            print(f"\nTesting nlist={nlist}")
            
            # Build IVF index
            build_start = time.time()
            
            dim = self.embeddings.shape[1]
            quantizer = faiss.IndexFlatIP(dim)
            index = faiss.IndexIVFFlat(quantizer, dim, nlist)
            
            index.train(self.embeddings)
            index.add(self.embeddings)
            
            build_time = time.time() - build_start
            
            # Test different nprobe values
            for nprobe in nprobe_values:
                if nprobe <= nlist:  # nprobe can't be larger than nlist
                    index.nprobe = nprobe
                    
                    # Measure query performance
                    query_times = []
                    recalls = []
                    
                    for i, query_emb in enumerate(query_embeddings):
                        start_time = time.time()
                        similarities, indices = index.search(query_emb.reshape(1, -1), 10)
                        query_time = time.time() - start_time
                        
                        query_times.append(query_time * 1000)
                        
                        # Calculate recall
                        predicted_indices = set(indices[0])
                        true_indices = set(ground_truth_results[i])
                        recall = len(predicted_indices.intersection(true_indices)) / len(true_indices)
                        recalls.append(recall)
                    
                    avg_query_time = np.mean(query_times)
                    avg_recall = np.mean(recalls)
                    
                    tuning_results.append({
                        'nlist': nlist,
                        'nprobe': nprobe,
                        'build_time': build_time,
                        'avg_query_time': avg_query_time,
                        'avg_recall': avg_recall,
                        'queries_per_second': 1000 / avg_query_time
                    })
                    
                    print(f"   nprobe={nprobe}: {avg_query_time:.2f}ms, recall={avg_recall:.3f}")
        
        return pd.DataFrame(tuning_results)

# Initialize parameter tuning lab
tuning_lab = ParameterTuningLab(embeddings, documents)
print("⚙️ Parameter Tuning Lab initialized!")

In [ ]:
# Run HNSW parameter tuning
hnsw_tuning_results = tuning_lab.tune_hnsw_parameters()

print("\n📊 HNSW TUNING RESULTS")
print("=" * 60)

# Show best configurations for different criteria
best_speed = hnsw_tuning_results.loc[hnsw_tuning_results['avg_query_time'].idxmin()]
best_accuracy = hnsw_tuning_results.loc[hnsw_tuning_results['avg_recall'].idxmax()]
best_balance = hnsw_tuning_results.loc[(hnsw_tuning_results['avg_recall'] * 1000 / hnsw_tuning_results['avg_query_time']).idxmax()]

print(f"🚀 Best Speed Configuration:")
print(f"   ef_construction={best_speed['ef_construction']}, M={best_speed['M']}, ef_search={best_speed['ef_search']}")
print(f"   Query time: {best_speed['avg_query_time']:.2f}ms, Recall: {best_speed['avg_recall']:.3f}")

print(f"\n🎯 Best Accuracy Configuration:")
print(f"   ef_construction={best_accuracy['ef_construction']}, M={best_accuracy['M']}, ef_search={best_accuracy['ef_search']}")
print(f"   Query time: {best_accuracy['avg_query_time']:.2f}ms, Recall: {best_accuracy['avg_recall']:.3f}")

print(f"\n⚖️ Best Balanced Configuration:")
print(f"   ef_construction={best_balance['ef_construction']}, M={best_balance['M']}, ef_search={best_balance['ef_search']}")
print(f"   Query time: {best_balance['avg_query_time']:.2f}ms, Recall: {best_balance['avg_recall']:.3f}")

# Visualize parameter effects
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# 1. Effect of ef_construction on build time
build_time_by_ef = hnsw_tuning_results.groupby('ef_construction')['build_time'].mean()
ax1.bar(build_time_by_ef.index, build_time_by_ef.values)
ax1.set_title('Effect of ef_construction on Build Time', fontweight='bold')
ax1.set_xlabel('ef_construction')
ax1.set_ylabel('Build Time (seconds)')

# 2. Effect of M on memory (approximate)
memory_by_M = hnsw_tuning_results.groupby('M').apply(lambda x: x['M'].iloc[0] * 4 * len(embeddings) / (1024**2))  # Rough estimate
ax2.bar(memory_by_M.index, memory_by_M.values)
ax2.set_title('Effect of M on Memory Usage (Estimated)', fontweight='bold')
ax2.set_xlabel('M (connections per node)')
ax2.set_ylabel('Estimated Memory (MB)')

# 3. Speed vs Accuracy trade-off
ax3.scatter(hnsw_tuning_results['avg_query_time'], hnsw_tuning_results['avg_recall'], 
           c=hnsw_tuning_results['ef_search'], cmap='viridis', s=50, alpha=0.7)
ax3.set_title('Speed vs Accuracy Trade-off (colored by ef_search)', fontweight='bold')
ax3.set_xlabel('Average Query Time (ms)')
ax3.set_ylabel('Average Recall')
colorbar = plt.colorbar(ax3.collections[0], ax=ax3)
colorbar.set_label('ef_search')

# 4. Effect of ef_search on performance
ef_search_perf = hnsw_tuning_results.groupby('ef_search').agg({
    'avg_query_time': 'mean',
    'avg_recall': 'mean'
})

ax4_twin = ax4.twinx()
bars = ax4.bar(ef_search_perf.index, ef_search_perf['avg_query_time'], alpha=0.7, label='Query Time')
line = ax4_twin.plot(ef_search_perf.index, ef_search_perf['avg_recall'], 'ro-', label='Recall')

ax4.set_title('Effect of ef_search on Performance', fontweight='bold')
ax4.set_xlabel('ef_search')
ax4.set_ylabel('Query Time (ms)', color='blue')
ax4_twin.set_ylabel('Recall', color='red')
ax4.legend(loc='upper left')
ax4_twin.legend(loc='upper right')

plt.tight_layout()
plt.show()

print("\n💡 HNSW Parameter Guidelines:")
print("- Higher ef_construction → Better accuracy, longer build time")
print("- Higher M → Better accuracy, more memory usage")
print("- Higher ef_search → Better accuracy, slower queries")
print("- Start with M=16, ef_construction=200, ef_search=100 for balanced performance")

In [ ]:
# Run IVF parameter tuning
ivf_tuning_results = tuning_lab.tune_ivf_parameters()

print("\n📊 IVF TUNING RESULTS")
print("=" * 60)

# Show best IVF configurations
best_ivf_speed = ivf_tuning_results.loc[ivf_tuning_results['avg_query_time'].idxmin()]
best_ivf_accuracy = ivf_tuning_results.loc[ivf_tuning_results['avg_recall'].idxmax()]

print(f"🚀 Best IVF Speed Configuration:")
print(f"   nlist={best_ivf_speed['nlist']}, nprobe={best_ivf_speed['nprobe']}")
print(f"   Query time: {best_ivf_speed['avg_query_time']:.2f}ms, Recall: {best_ivf_speed['avg_recall']:.3f}")

print(f"\n🎯 Best IVF Accuracy Configuration:")
print(f"   nlist={best_ivf_accuracy['nlist']}, nprobe={best_ivf_accuracy['nprobe']}")
print(f"   Query time: {best_ivf_accuracy['avg_query_time']:.2f}ms, Recall: {best_ivf_accuracy['avg_recall']:.3f}")

# Visualize IVF parameter effects
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# 1. Heatmap of nlist vs nprobe performance
pivot_time = ivf_tuning_results.pivot(index='nlist', columns='nprobe', values='avg_query_time')
sns.heatmap(pivot_time, annot=True, fmt='.2f', ax=ax1, cmap='YlOrRd')
ax1.set_title('Query Time (ms) by nlist and nprobe', fontweight='bold')

# 2. Heatmap of recall
pivot_recall = ivf_tuning_results.pivot(index='nlist', columns='nprobe', values='avg_recall')
sns.heatmap(pivot_recall, annot=True, fmt='.3f', ax=ax2, cmap='YlGnBu')
ax2.set_title('Recall by nlist and nprobe', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n💡 IVF Parameter Guidelines:")
print("- Higher nlist → More clusters, better accuracy but slower build")
print("- Higher nprobe → Search more clusters, better accuracy but slower queries")
print("- Rule of thumb: nlist ≈ sqrt(N), nprobe ≈ nlist/10")
print("- For 5K vectors: nlist=100, nprobe=10 provides good balance")

## 🔄 Exercise 4: Index Maintenance and Dynamic Updates

Let's explore index maintenance strategies and dynamic update scenarios.

In [ ]:
class IndexMaintenanceLab:
    """Laboratory for index maintenance and update strategies"""
    
    def __init__(self):
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
        self.performance_history = []
    
    def simulate_dynamic_updates(self, initial_size: int = 1000, 
                                 update_batches: int = 5, batch_size: int = 200) -> Dict[str, Any]:
        """Simulate dynamic index updates and measure performance degradation"""
        print("🔄 DYNAMIC INDEX UPDATE SIMULATION")
        print("=" * 60)
        
        # Generate initial dataset
        print(f"Generating initial dataset: {initial_size} vectors")
        initial_docs = [f"Initial document {i} about machine learning concepts" for i in range(initial_size)]
        initial_embeddings = self.embedding_model.encode(initial_docs)
        initial_embeddings = initial_embeddings / np.linalg.norm(initial_embeddings, axis=1, keepdims=True)
        
        # Build initial HNSW index
        dim = initial_embeddings.shape[1]
        hnsw_index = hnswlib.Index(space='cosine', dim=dim)
        hnsw_index.init_index(max_elements=initial_size + update_batches * batch_size, ef_construction=200, M=16)
        hnsw_index.add_items(initial_embeddings, np.arange(initial_size))
        
        # Test queries
        test_queries = [
            "machine learning algorithms and methods",
            "data science and analytics",
            "artificial intelligence research"
        ]
        query_embeddings = self.embedding_model.encode(test_queries)
        query_embeddings = query_embeddings / np.linalg.norm(query_embeddings, axis=1, keepdims=True)
        
        simulation_results = []
        current_size = initial_size
        
        # Measure initial performance
        initial_perf = self._measure_query_performance(hnsw_index, query_embeddings)
        simulation_results.append({
            'stage': 'initial',
            'total_vectors': current_size,
            'avg_query_time': initial_perf['avg_time'],
            'p95_query_time': initial_perf['p95_time'],
            'recall': 1.0  # Assume perfect recall for initial state
        })
        
        print(f"Initial performance: {initial_perf['avg_time']:.2f}ms avg, {initial_perf['p95_time']:.2f}ms p95")
        
        # Simulate incremental updates
        for batch_num in range(update_batches):
            print(f"\nAdding batch {batch_num + 1}: {batch_size} new vectors")
            
            # Generate new documents
            new_docs = [f"New document {current_size + i} about advanced topics" for i in range(batch_size)]
            new_embeddings = self.embedding_model.encode(new_docs)
            new_embeddings = new_embeddings / np.linalg.norm(new_embeddings, axis=1, keepdims=True)
            
            # Add to index
            start_time = time.time()
            new_labels = np.arange(current_size, current_size + batch_size)
            hnsw_index.add_items(new_embeddings, new_labels)
            update_time = time.time() - start_time
            
            current_size += batch_size
            
            # Measure performance after update
            perf = self._measure_query_performance(hnsw_index, query_embeddings)
            
            simulation_results.append({
                'stage': f'after_batch_{batch_num + 1}',
                'total_vectors': current_size,
                'avg_query_time': perf['avg_time'],
                'p95_query_time': perf['p95_time'],
                'update_time': update_time,
                'recall': 0.95  # Assume some degradation
            })
            
            print(f"   Update time: {update_time:.3f}s")
            print(f"   New performance: {perf['avg_time']:.2f}ms avg, {perf['p95_time']:.2f}ms p95")
        
        return {
            'simulation_results': simulation_results,
            'final_index': hnsw_index,
            'total_vectors': current_size
        }
    
    def _measure_query_performance(self, index: hnswlib.Index, query_embeddings: np.ndarray) -> Dict[str, float]:
        """Measure query performance on an index"""
        query_times = []
        
        index.set_ef(100)  # Standard ef value
        
        for query_emb in query_embeddings:
            start_time = time.time()
            labels, distances = index.knn_query(query_emb, k=10)
            query_time = time.time() - start_time
            query_times.append(query_time * 1000)  # Convert to ms
        
        return {
            'avg_time': np.mean(query_times),
            'p95_time': np.percentile(query_times, 95),
            'p99_time': np.percentile(query_times, 99)
        }
    
    def compare_rebuild_vs_incremental(self, base_size: int = 1000, addition_size: int = 500) -> Dict[str, Any]:
        """Compare rebuilding entire index vs incremental updates"""
        print("\n🔄 REBUILD vs INCREMENTAL UPDATE COMPARISON")
        print("=" * 60)
        
        # Generate datasets
        base_docs = [f"Base document {i} about technology" for i in range(base_size)]
        additional_docs = [f"Additional document {i} about innovation" for i in range(addition_size)]
        
        base_embeddings = self.embedding_model.encode(base_docs)
        additional_embeddings = self.embedding_model.encode(additional_docs)
        
        # Normalize
        base_embeddings = base_embeddings / np.linalg.norm(base_embeddings, axis=1, keepdims=True)
        additional_embeddings = additional_embeddings / np.linalg.norm(additional_embeddings, axis=1, keepdims=True)
        
        all_embeddings = np.vstack([base_embeddings, additional_embeddings])
        
        dim = base_embeddings.shape[1]
        
        # Strategy 1: Incremental Update
        print("\n1. Incremental Update Strategy:")
        
        # Build initial index
        start_time = time.time()
        incremental_index = hnswlib.Index(space='cosine', dim=dim)
        incremental_index.init_index(
            max_elements=base_size + addition_size, 
            ef_construction=200, 
            M=16
        )
        incremental_index.add_items(base_embeddings, np.arange(base_size))
        initial_build_time = time.time() - start_time
        
        # Add new items incrementally
        start_time = time.time()
        incremental_index.add_items(additional_embeddings, np.arange(base_size, base_size + addition_size))
        incremental_update_time = time.time() - start_time
        
        total_incremental_time = initial_build_time + incremental_update_time
        
        print(f"   Initial build: {initial_build_time:.3f}s")
        print(f"   Incremental update: {incremental_update_time:.3f}s")
        print(f"   Total time: {total_incremental_time:.3f}s")
        
        # Strategy 2: Full Rebuild
        print("\n2. Full Rebuild Strategy:")
        
        start_time = time.time()
        rebuild_index = hnswlib.Index(space='cosine', dim=dim)
        rebuild_index.init_index(
            max_elements=base_size + addition_size,
            ef_construction=200,
            M=16
        )
        rebuild_index.add_items(all_embeddings, np.arange(len(all_embeddings)))
        rebuild_time = time.time() - start_time
        
        print(f"   Full rebuild: {rebuild_time:.3f}s")
        
        # Compare query performance
        test_queries = [
            "technology and innovation",
            "modern solutions",
            "advanced methods"
        ]
        query_embeddings = self.embedding_model.encode(test_queries)
        query_embeddings = query_embeddings / np.linalg.norm(query_embeddings, axis=1, keepdims=True)
        
        incremental_perf = self._measure_query_performance(incremental_index, query_embeddings)
        rebuild_perf = self._measure_query_performance(rebuild_index, query_embeddings)
        
        print("\n📊 Performance Comparison:")
        print(f"Incremental - Avg: {incremental_perf['avg_time']:.2f}ms, P95: {incremental_perf['p95_time']:.2f}ms")
        print(f"Rebuild     - Avg: {rebuild_perf['avg_time']:.2f}ms, P95: {rebuild_perf['p95_time']:.2f}ms")
        
        print("\n⏱️ Time Comparison:")
        print(f"Incremental: {total_incremental_time:.3f}s")
        print(f"Rebuild:     {rebuild_time:.3f}s")
        print(f"Speedup:     {rebuild_time / total_incremental_time:.2f}x {'(incremental faster)' if total_incremental_time < rebuild_time else '(rebuild faster)'}")
        
        return {
            'incremental_time': total_incremental_time,
            'rebuild_time': rebuild_time,
            'incremental_perf': incremental_perf,
            'rebuild_perf': rebuild_perf
        }
    
    def recommend_maintenance_strategy(self, dataset_size: int, update_frequency: str, 
                                      accuracy_requirement: str) -> Dict[str, str]:
        """Recommend maintenance strategy based on requirements"""
        recommendations = {}
        
        # Base recommendations
        if update_frequency == "high" and dataset_size < 100000:
            recommendations["strategy"] = "incremental_updates"
            recommendations["reason"] = "High update frequency with moderate dataset size favors incremental updates"
        elif dataset_size > 1000000 and update_frequency == "low":
            recommendations["strategy"] = "scheduled_rebuild"
            recommendations["reason"] = "Large datasets with infrequent updates benefit from periodic full rebuilds"
        else:
            recommendations["strategy"] = "hybrid_approach"
            recommendations["reason"] = "Mixed requirements suggest combination of incremental updates with periodic rebuilds"
        
        # Accuracy considerations
        if accuracy_requirement == "high":
            recommendations["accuracy_note"] = "Consider more frequent rebuilds or higher HNSW parameters"
        
        # Specific parameters
        if dataset_size < 10000:
            recommendations["hnsw_params"] = "M=16, ef_construction=200, ef_search=100"
        elif dataset_size < 100000:
            recommendations["hnsw_params"] = "M=16, ef_construction=200, ef_search=50"
        else:
            recommendations["hnsw_params"] = "M=32, ef_construction=400, ef_search=100"
        
        return recommendations

# Initialize index maintenance lab
maintenance_lab = IndexMaintenanceLab()
print("🔄 Index Maintenance Lab initialized!")

In [ ]:
# Run dynamic update simulation
update_simulation = maintenance_lab.simulate_dynamic_updates(
    initial_size=800, 
    update_batches=4, 
    batch_size=150
)

# Visualize performance degradation over time
sim_results = update_simulation['simulation_results']
sim_df = pd.DataFrame(sim_results)

print("\n📊 DYNAMIC UPDATE PERFORMANCE")
print("=" * 50)
print(sim_df.to_string(index=False, float_format='%.3f'))

# Plot performance over time
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Query time vs dataset size
ax1.plot(sim_df['total_vectors'], sim_df['avg_query_time'], 'bo-', label='Average Query Time')
ax1.plot(sim_df['total_vectors'], sim_df['p95_query_time'], 'ro-', label='P95 Query Time')
ax1.set_title('Query Performance vs Dataset Size', fontweight='bold')
ax1.set_xlabel('Total Vectors')
ax1.set_ylabel('Query Time (ms)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Update times
update_times = [result.get('update_time', 0) for result in sim_results[1:]]  # Skip initial
batch_numbers = list(range(1, len(update_times) + 1))
ax2.bar(batch_numbers, update_times)
ax2.set_title('Incremental Update Times', fontweight='bold')
ax2.set_xlabel('Update Batch')
ax2.set_ylabel('Update Time (seconds)')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Dynamic Update Insights:")
print("- Query performance slightly degrades as dataset grows")
print("- HNSW handles incremental updates efficiently")
print("- Update times remain relatively constant per batch")
print("- Consider rebuilding when performance degrades significantly")

In [ ]:
# Compare rebuild vs incremental strategies
comparison_results = maintenance_lab.compare_rebuild_vs_incremental(
    base_size=800, 
    addition_size=400
)

# Test different scenarios and get recommendations
print("\n🎯 MAINTENANCE STRATEGY RECOMMENDATIONS")
print("=" * 60)

scenarios = [
    {"size": 10000, "frequency": "high", "accuracy": "medium", "name": "Small Dataset, Frequent Updates"},
    {"size": 100000, "frequency": "medium", "accuracy": "high", "name": "Medium Dataset, High Accuracy"},
    {"size": 1000000, "frequency": "low", "accuracy": "medium", "name": "Large Dataset, Infrequent Updates"},
    {"size": 50000, "frequency": "high", "accuracy": "high", "name": "High Throughput, High Accuracy"}
]

for scenario in scenarios:
    print(f"\n📋 Scenario: {scenario['name']}")
    print(f"   Dataset size: {scenario['size']:,} vectors")
    print(f"   Update frequency: {scenario['frequency']}")
    print(f"   Accuracy requirement: {scenario['accuracy']}")
    
    recommendations = maintenance_lab.recommend_maintenance_strategy(
        scenario['size'], scenario['frequency'], scenario['accuracy']
    )
    
    print(f"   ✅ Recommended strategy: {recommendations['strategy']}")
    print(f"   📝 Reason: {recommendations['reason']}")
    print(f"   ⚙️ HNSW parameters: {recommendations['hnsw_params']}")
    if 'accuracy_note' in recommendations:
        print(f"   🎯 Accuracy note: {recommendations['accuracy_note']}")

print("\n📈 GENERAL MAINTENANCE GUIDELINES")
print("=" * 60)
print("1. 🔄 Incremental Updates:")
print("   - Best for: Frequent updates, real-time systems")
print("   - Drawback: Gradual performance degradation")
print("   - Monitor: Query latency trends")

print("\n2. 🏗️ Full Rebuilds:")
print("   - Best for: Batch processing, optimal performance")
print("   - Drawback: Temporary downtime, resource intensive")
print("   - Schedule: Off-peak hours, based on degradation")

print("\n3. 🔀 Hybrid Approach:")
print("   - Incremental updates for daily operations")
print("   - Scheduled rebuilds weekly/monthly")
print("   - Monitor performance metrics to adjust frequency")

print("\n4. 📊 Monitoring Metrics:")
print("   - P95 query latency < 100ms")
print("   - Recall@10 > 0.95")
print("   - Index fragmentation ratio")
print("   - Memory usage growth rate")

## 🎯 Key Takeaways

From this module, you should now understand:

### 🏗️ Indexing Algorithm Characteristics:
1. **HNSW**: Best overall performance, moderate memory usage, excellent for most use cases
2. **IVF**: Good for large datasets, memory efficient, requires parameter tuning
3. **LSH**: Extremely fast but lower accuracy, good for massive datasets
4. **Product Quantization**: Dramatically reduces memory usage with some accuracy trade-off

### ⚖️ Performance Trade-offs:
1. **Speed vs Accuracy**: Approximate algorithms trade precision for speed
2. **Memory vs Performance**: More memory usually enables faster queries
3. **Build Time vs Query Time**: Complex indexes take longer to build but query faster
4. **Update Frequency vs Optimization**: Frequent updates may require different strategies

### 🔧 Parameter Tuning Guidelines:

#### HNSW Parameters:
- **ef_construction**: Higher = better accuracy, longer build (start with 200)
- **M**: Higher = better accuracy, more memory (start with 16)
- **ef_search**: Higher = better accuracy, slower queries (start with 100)

#### IVF Parameters:
- **nlist**: Number of clusters ≈ sqrt(dataset_size)
- **nprobe**: Clusters to search ≈ nlist/10

### 🔄 Index Maintenance Strategies:
1. **Incremental Updates**: Good for real-time systems with frequent updates
2. **Full Rebuilds**: Optimal performance but requires downtime
3. **Hybrid Approach**: Combine both based on monitoring metrics
4. **Monitoring**: Track P95 latency, recall, memory usage, and fragmentation

### 📊 Performance Targets (2025 Standards):
- **Query Latency**: <50ms P95 for production systems
- **Recall Rate**: >0.95 for most applications
- **Memory Usage**: <2x original embedding size
- **Build Time**: <1 hour for 10M vectors

### 🚀 Optimization Workflow:
1. **Start with defaults** → 2. **Measure baseline** → 3. **Identify bottlenecks** → 4. **Tune parameters** → 5. **Validate improvements** → 6. **Monitor in production**

## 🎯 Next Steps

In the next modules, we'll explore:
- **Module 8**: Comparing different search methods (semantic vs lexical vs hybrid)
- **Module 9**: Advanced retrieval strategies and re-ranking techniques
- **Module 10**: Prompt engineering for optimal RAG performance

Mastering indexing and performance optimization is crucial for building production-ready RAG systems that can handle millions of documents with sub-50ms query latencies!

## 🤔 Discussion Questions

1. When would you choose IVF over HNSW for your vector database?
2. How would you handle a scenario where your dataset grows from 10K to 10M vectors?
3. What monitoring metrics would you implement for a production vector search system?
4. How would you balance update frequency with query performance in a real-time system?
5. What factors would influence your decision to rebuild an index vs continue with incremental updates?

## 📝 Optional Exercises

1. **GPU Acceleration**: Experiment with FAISS-GPU for larger datasets
2. **Memory Profiling**: Implement detailed memory usage tracking during index operations
3. **Production Monitoring**: Build a dashboard to track index performance metrics
4. **Custom Indexing**: Implement a simple LSH algorithm from scratch
5. **A/B Testing**: Design experiments to compare indexing strategies in production